---
### **4. folium basic**
- folium (https://python-visualization.github.io/folium/)
- folium은 python에서 interactive map으로 가장 많이 쓰이는 라이브러리 입니다. 

In [3]:
### 설치 
!pip install folium

  Using cached folium-0.14.0-py2.py3-none-any.whl (102 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached branca-0.6.0-py3-none-any.whl (24 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
     -------------------------------------- 123.6/123.6 kB 7.1 MB/s eta 0:00:00
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached charset_normalizer-3.1.0-cp38-cp38-win_amd64.whl (96 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.4.0 requires absl-py, which is not installed.
tensorflow-datasets 4.4.0 requires future, which is not installed.
tensorflow-datasets 4.4.0 requires promise, which is not installed.
tensorflow-datasets 4.4.0 requires protobuf>=3.12.2, which is not installed.
tensorflow-datasets 4.4.0 requires termcolor, which is not installed.
tensorflow-datasets 4.4.0 requires tqdm, which is not installed.
spacy 2.3.7 requires tqdm<5.0.0,>=4.38.0, which is not installed.
ray 1.13.0 requires aiosignal, which is not installed.
ray 1.13.0 requires filelock, which is not installed.
ray 1.13.0 requires frozenlist, which is not installed.
ray 1.13.0 requires grpcio<=1.43.0,>=1.28.1, which is not installed.
ray 1.13.0 requires jsonschema, which is not installed.
ray 1.13.0 requires msgpack<2.0.0,>=1.0.0, which is no

In [1]:
### library 
import folium
import geopandas as gpd
import warnings 

warnings.filterwarnings('ignore')

---
#### **4-1. 기본 map 생성 및 저장**

- **기본 map 생성**

In [2]:
###  folium.Map(location=[위도, 경도], zoom_start=시작 확대 정도) 
m = folium.Map(location=[36.3398175, 127.3940486], zoom_start=12) # 대전 중심점으로 기본 지도 생성 
m

- **Map style**

In [3]:
### openstreetmap (default)
m = folium.Map(location=[36.3398175, 127.3940486], tiles='openstreetmap', zoom_start=12)
m

In [4]:
### stamenterrain 
m = folium.Map(location=[36.3398175, 127.3940486], tiles='stamenterrain', zoom_start=12)
m

In [5]:
### cartodbpositron 
m = folium.Map(location=[36.3398175, 127.3940486], tiles='cartodbpositron', zoom_start=12)
m

In [6]:
### cartodbdark_matter 
m = folium.Map(location=[36.3398175, 127.3940486], tiles='cartodbdark_matter', zoom_start=12)
m

- **map 저장**

In [19]:
###  folium.Map(location=[위도, 경도], zoom_start=시작 확대 정도) 
m = folium.Map(location=[36.3398175, 127.3940486], zoom_start=12) # 대전 중심점으로 기본 지도 생성 
m.save('base_map.html')

---
#### **4-2. Marker**

In [7]:
tasu_station = gpd.read_file('./data/Lecture3/tasu_station_inf.shp')
tasu_station.head(2)

ID      stationNam  holder   address                    geometry
0   1  무역전시관입구(택시승강장)      14   도룡동 3-8  POINT (127.38613 36.37458)
1   2         대전컨벤션센터      20  도룡동 4-19  POINT (127.39238 36.37439)

In [8]:
### 지도를 그릴 때는 base map 부터 설정한 후 시각화를 진행한다. 
m = folium.Map(location=[36.3398175, 127.3940486], zoom_start=12) 

### Marker 
# - folium.Marker([위도, 경도]).add_to(베이스 맵)
for idx, row in tasu_station.iterrows():
    folium.Marker([row['geometry'].y, row['geometry'].x]).add_to(m) 

m

---
#### **4-3. Circle**


In [9]:
### 지도를 그릴 때는 base map 부터 설정한 후 시각화를 진행한다. 
m = folium.Map(location=[36.3398175, 127.3940486], zoom_start=12) 

### Circle 
# - folium.Circle([위도, 경도]).add_to(베이스 맵)
# - parameter = {radius : 반경, color = 색깔}
for idx, row in tasu_station.iterrows():
    folium.Circle([row['geometry'].y, row['geometry'].x], 
                  radius=20,
                  color='red').add_to(m) 

m

---
#### **4-4. Choropleth**

In [10]:
HJD = gpd.read_file('./data/Lecture3/HangJeongDong.geojson')
daejeon_HJD = HJD.loc[(HJD['sidonm'] == '대전광역시')].reset_index(drop=True)

In [11]:
### 지도를 그릴 때는 base map 부터 설정한 후 시각화를 진행한다. 
m = folium.Map(location=[36.3398175, 127.3940486], zoom_start=11) 

### Choropleth 
folium.Choropleth(geo_data=daejeon_HJD['geometry']).add_to(m) 

m

---
#### **4-5. 행정구역 별 타슈 정류장 시각화**

- **방법 1**

In [12]:
m = folium.Map(location=[36.3398175, 127.3940486], zoom_start=11) 

folium.Choropleth(geo_data=daejeon_HJD['geometry']).add_to(m) 
for idx, row in tasu_station.iterrows():
    folium.Circle([row['geometry'].y, row['geometry'].x], 
                  radius=20,
                  color='red').add_to(m) 

m

- **방법 2**

In [13]:
daejeon_tasu_count = gpd.sjoin(daejeon_HJD, tasu_station, how='inner', predicate='intersects')
daejeon_tasu_count_dict = daejeon_tasu_count.value_counts('temp').to_dict()
daejeon_HJD['tasuStation_count'] = [daejeon_tasu_count_dict[i] if i in daejeon_tasu_count_dict.keys() else 0 for i in daejeon_HJD['temp']]
daejeon_HJD.head(2)

OBJECTID         adm_nm   adm_cd     adm_cd2    sgg sido sidonm     temp  \
0      1008    대전광역시 동구 효동  2501053  3011053000  30110   30  대전광역시    동구 효동   
1      1009  대전광역시 동구 판암1동  2501055  3011055100  30110   30  대전광역시  동구 판암1동   

  sggnm   adm_cd8                                           geometry  \
0    동구  25010530  MULTIPOLYGON (((127.44846 36.31925, 127.44832 ...   
1    동구  25010550  MULTIPOLYGON (((127.48100 36.30044, 127.47953 ...   

   tasuStation_count  
0                  6  
1                  3

In [14]:
m = folium.Map(location=[36.3398175, 127.3940486], zoom_start=11) 

folium.Choropleth(geo_data=daejeon_HJD['geometry'],
                  data=daejeon_HJD['tasuStation_count'],
                  key_on='feature.id',
                  fill_color='YlGnBu',
                  legend_name='타슈 정거장 갯수').add_to(m) 

m

---